<a href="https://colab.research.google.com/github/skaurl/P4-Model-Optimization/blob/main/Mask_Classfier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tflite-model-maker

In [ ]:
import tarfile

f = tarfile.open('/content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/train.tar.gz')
f.extractall()
f.close()

In [ ]:
!find . -type f -name '._*' -delete

In [ ]:
import os

import pandas as pd
import numpy as np

from tqdm import tqdm
from glob import glob
import shutil
import cv2
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

In [ ]:
os.makedirs('/content/train_dataset')
os.makedirs('/content/train_dataset/mask')
os.makedirs('/content/train_dataset/no_mask')
os.makedirs('/content/train_dataset/background')

os.makedirs('/content/val_dataset')
os.makedirs('/content/val_dataset/mask')
os.makedirs('/content/val_dataset/no_mask')
os.makedirs('/content/val_dataset/background')

In [ ]:
img_info = pd.read_csv('/content/train/train.csv')

img_info_train, img_info_val = train_test_split(img_info, test_size=0.1, random_state=42)

In [ ]:
for num in tqdm(range(len(img_info_train))):
    for name in os.listdir('/content/train/images/'+img_info_train.iloc[num,4]):
        original_path = '/content/train/images/'+img_info_train.iloc[num,4]+'/'+name
        destination_path = '/content/train_dataset/'

        if name[:4] == 'mask':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'mask/'+img_info_train.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'mask/'+img_info_train.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'mask/'+img_info_train.iloc[num,4]+'_c_'+name,img)
        elif name[:4] == 'inco':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_c_'+name,img)
        elif name[:4] == 'norm':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_c_'+name,img)
            
            img = cv2.imread(original_path)[:128,:128]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-64:256+64,:128]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[-128:,:128]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_c_'+name,img)
            img = cv2.imread(original_path)[:128,192-64:192+64]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_d_'+name,img)
            img = cv2.imread(original_path)[-128:,192-64:192+64]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_e_'+name,img)
            img = cv2.imread(original_path)[:128,-128:]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_f_'+name,img)
            img = cv2.imread(original_path)[256-64:256+64,-128:]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_g_'+name,img)
            img = cv2.imread(original_path)[-128:,-128:]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_h_'+name,img)

In [ ]:
for num in tqdm(range(len(img_info_val))):
    for name in os.listdir('/content/train/images/'+img_info_val.iloc[num,4]):
        original_path = '/content/train/images/'+img_info_val.iloc[num,4]+'/'+name
        destination_path = '/content/val_dataset/'

        if name[:4] == 'mask':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'mask/'+img_info_val.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'mask/'+img_info_val.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'mask/'+img_info_val.iloc[num,4]+'_c_'+name,img)
        elif name[:4] == 'inco':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_c_'+name,img)
        elif name[:4] == 'norm':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_c_'+name,img)
            
            img = cv2.imread(original_path)[:128,:128]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-64:256+64,:128]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[-128:,:128]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_c_'+name,img)
            img = cv2.imread(original_path)[:128,192-64:192+64]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_d_'+name,img)
            img = cv2.imread(original_path)[-128:,192-64:192+64]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_e_'+name,img)
            img = cv2.imread(original_path)[:128,-128:]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_f_'+name,img)
            img = cv2.imread(original_path)[256-64:256+64,-128:]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_g_'+name,img)
            img = cv2.imread(original_path)[-128:,-128:]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_h_'+name,img)

In [ ]:
train_data = DataLoader.from_folder('/content/train_dataset')
val_data = DataLoader.from_folder('/content/val_dataset')

In [ ]:
custom = image_classifier.ModelSpec(uri='https://tfhub.dev/google/imagenet/mobilenet_v1_025_128/classification/5')

In [ ]:
model = image_classifier.create(train_data=train_data,
                                model_spec=custom,
                                validation_data=val_data,
                                batch_size=512,
                                epochs=50,
                                learning_rate=5e-4,
                                shuffle=True,
                                )

In [ ]:
config = QuantizationConfig.for_dynamic()

In [ ]:
model.export(export_dir='/content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/', quantization_config=config)

In [ ]:
model.evaluate_tflite('/content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/model.tflite', val_data)